<a href="https://colab.research.google.com/github/SyedHasnat/CNNs-LSTMs-Time-Series/blob/main/Copy_of_Hybrid_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/0_Hasnat/pyimagesearch')
from timeseires.CNN1D.GoogLeNet1D import PC
from timeseires.utils.to_split import to_split
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import pydot
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Dense, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate
from tensorflow.keras.regularizers import l2


In [4]:
tensorflow.__version__

'2.8.2'

In [5]:
path_tr = '/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Datset/8_train.csv'
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:,[1, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]].values 
path_v = '/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Datset/8_validation.csv'
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:,[1, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]].values 
path_te = '/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Datset/8_test.csv'
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:,[1, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]].values
#.......................................................................................................
scaler = pickle.load(open("/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Datset/scaler.pkl", 'rb'))


In [6]:
time_steps=72
num_features=17

In [7]:
start = time.time()
train_X , train_y = to_split(train_set, time_steps=time_steps, target_col=0, target_len=1)
validation_X, validation_y = to_split(validation_set, time_steps=time_steps, target_col=0, target_len=1)
test_X, test_y = to_split(test_set, time_steps=time_steps, target_col=0, target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.6039564609527588 sec


In [8]:
model = None
start_epoch = 0

**E2**

In [9]:
class PC:
    @staticmethod
    def conv_module(x, num_filter, kr, stride ,padding="same", reg=0.0005, name=None):
        # initialize the CONV, BN, and RELU layer names
        (convName, bnName, actName) = (None, None, None)
        
        # if a layer name was supplied, prepend it
        if name is not None:
            convName = name + "_conv"
            bnName = name + "_bn"
            actName = name + "_act"
        # define a CONV => BN => RELU pattern
        x = Conv1D(num_filter, kr, strides=stride, padding=padding, kernel_regularizer=l2(reg),name=convName)(x)
        x = BatchNormalization(axis=-1, name=bnName)(x)
        x = Activation("relu", name=actName)(x)
        
        # return the block
        return x
    
    @staticmethod
    def inception_module(x, num1x1, num3x3Reduce, num3x3,num5x5Reduce, num5x5, stage, reg=0.0005):
        # define the first branch of the Inception module which
        # consists of 1x1 convolutions
        first = PC.conv_module(x, num1x1, 1, 1, reg=reg, name=stage + "_first")
        
        # define the second branch of the Inception module which
        # consists of 1x1 and 3x3 convolutions
        second = PC.conv_module(x, num3x3Reduce, 1, 1,  reg=reg, name=stage + "_second1")
        second = PC.conv_module(second, num3x3, 3, 1, reg=reg, name=stage + "_second2")
        
        # define the third branch of the Inception module which
        # are our 1x1 and 5x5 convolutions
        third = PC.conv_module(x, num5x5Reduce, 1, 1,  reg=reg, name=stage + "_third1")
        third = PC.conv_module(third, num5x5, 5, 1,  reg=reg, name=stage + "_third2")
        
        # concatenate across the channel dimension
        x = concatenate([first, second, third], axis=-1 ,name=stage + "_mixed")
        
        # return the block
        return x
    @staticmethod
    def build(time_steps, num_features, reg=0.0005):
        # initialize the input shape to be "channels last" and the
        # channels dimension itself
        # define the model input and first CONV module
        inputs = Input(shape=(time_steps, num_features))
        x = LSTM(48, return_sequences=True)(inputs)
        x = Activation('relu')(x)
        x = PC.conv_module(x, 32, 7, 1)
        x = PC.conv_module(x, 64, 3, 1)
        x = PC.inception_module(x, 32, 32, 64, 8, 32, "2a")
        x = MaxPooling1D(2)(x)
        x = PC.inception_module(x, 64, 64, 128, 16, 64, "3a")
#.......................................................................
        x = Flatten()(x)
        x = Dense(10, activation='relu')(x)
        x = Dense(1)(x)
#.......................................................................
        # create the model
        model = Model(inputs, x, name="PC")
        
        # return the constructed network architecture
        return model


In [10]:
checkpoints = '/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = '/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"historyE2.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"historyE2.json"])

In [11]:
model = None
start_epoch = 0
time_steps=72
num_features=17

In [12]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [13]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=72, num_features=17, reg=0.0005)
    opt = Adam(1e-2)
    model.compile(loss= 'mae', optimizer='adam', metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-3)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [14]:
epochs = 200
verbose = 1 #0
batch_size = 1024
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)


Epoch 1/200
32/32 [==============================] - ETA: 0s - loss: 0.6664 - mae: 0.3937 - mape: 9542.7539
Epoch 1: val_loss improved from inf to 0.77432, saving model to /content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2-cp-0001-loss0.77.h5
32/32 [==============================] - 64s 2s/step - loss: 0.6664 - mae: 0.3937 - mape: 9542.7539 - val_loss: 0.7743 - val_mae: 0.5052 - val_mape: 82.5226
Epoch 2/200
32/32 [==============================] - ETA: 0s - loss: 0.3164 - mae: 0.0519 - mape: 8677.2988
Epoch 2: val_loss improved from 0.77432 to 0.66819, saving model to /content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2-cp-0002-loss0.67.h5
32/32 [==============================] - 58s 2s/step - loss: 0.3164 - mae: 0.0519 - mape: 8677.2988 - val_loss: 0.6682 - val_mae: 0.4092 - val_mape: 64.9635
Epoch 3/200
32/32 [==============================] - ETA: 0s - loss: 0.2935 - mae: 0.0400 

In [15]:
model = load_model('/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2-cp-0192-loss0.02.h5')
y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y.reshape(-1, 1))
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled))
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 235.89
Median Absolute Error (MedAE): 175.39
Mean Squared Error (MSE): 98764.13
Root Mean Squared Error (RMSE): 314.27
Mean Absolute Percentage Error (MAPE): 2.28 %
Median Absolute Percentage Error (MDAPE): 1.75 %


y_test_unscaled.shape=  (4600, 1)
y_pred.shape=  (4600, 1)


**E2T2**

In [16]:
checkpoints = '/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2t2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model='/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2-cp-0192-loss0.02.h5'
start_epoch=193

In [17]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [18]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=72, num_features=17, reg=0.0005)
    opt = Adam(1e-2)
    model.compile(loss= 'mae', optimizer='adam', metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading /content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2-cp-0192-loss0.02.h5...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [19]:
epochs = 40
verbose = 1 #0
batch_size = 1024
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)


Epoch 1/40
32/32 [==============================] - ETA: 0s - loss: 0.0127 - mae: 0.0113 - mape: 8952.0029
Epoch 1: val_loss improved from inf to 0.03552, saving model to /content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2t2-cp-0001-loss0.04.h5
32/32 [==============================] - 63s 2s/step - loss: 0.0127 - mae: 0.0113 - mape: 8952.0029 - val_loss: 0.0355 - val_mae: 0.0341 - val_mape: 5.5727
Epoch 2/40
32/32 [==============================] - ETA: 0s - loss: 0.0124 - mae: 0.0110 - mape: 8941.2197
Epoch 2: val_loss did not improve from 0.03552
32/32 [==============================] - 57s 2s/step - loss: 0.0124 - mae: 0.0110 - mape: 8941.2197 - val_loss: 0.0487 - val_mae: 0.0474 - val_mape: 7.6946
Epoch 3/40
32/32 [==============================] - ETA: 0s - loss: 0.0119 - mae: 0.0105 - mape: 8997.8818
Epoch 3: val_loss did not improve from 0.03552
32/32 [==============================] - 56s 2s/step - loss: 0.0119 - mae: 0.0105 -

In [20]:
model = load_model('/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2t2-cp-0037-loss0.03.h5')
y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y.reshape(-1, 1))
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled))
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 371.33
Median Absolute Error (MedAE): 327.53
Mean Squared Error (MSE): 204001.56
Root Mean Squared Error (RMSE): 451.67
Mean Absolute Percentage Error (MAPE): 3.51 %
Median Absolute Percentage Error (MDAPE): 3.24 %


y_test_unscaled.shape=  (4600, 1)
y_pred.shape=  (4600, 1)


**E2T3**

In [21]:
checkpoints = '/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/CheckpointsE3/E2t3-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model='/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2t2-cp-0037-loss0.03.h5'
start_epoch=230 #from curve

In [22]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [23]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=72, num_features=17, reg=0.0005)
    opt = Adam(1e-2)
    model.compile(loss= 'mae', optimizer='adam', metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-7)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading /content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/checkpointsE3/E2t2-cp-0037-loss0.03.h5...
[INFO] old learning rate: 9.999999747378752e-05
[INFO] new learning rate: 1.0000000116860974e-07


In [24]:
epochs = 40
verbose = 1 #0
batch_size = 1024
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)


Epoch 1/40
32/32 [==============================] - ETA: 0s - loss: 0.0117 - mae: 0.0107 - mape: 9082.8115
Epoch 1: val_loss improved from inf to 0.02162, saving model to /content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/CheckpointsE3/E2t3-cp-0001-loss0.02.h5
32/32 [==============================] - 61s 2s/step - loss: 0.0117 - mae: 0.0107 - mape: 9082.8115 - val_loss: 0.0216 - val_mae: 0.0206 - val_mape: 3.4650
Epoch 2/40
32/32 [==============================] - ETA: 0s - loss: 0.0111 - mae: 0.0101 - mape: 9109.3252
Epoch 2: val_loss improved from 0.02162 to 0.01808, saving model to /content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/CheckpointsE3/E2t3-cp-0002-loss0.02.h5
32/32 [==============================] - 58s 2s/step - loss: 0.0111 - mae: 0.0101 - mape: 9109.3252 - val_loss: 0.0181 - val_mae: 0.0171 - val_mape: 2.9254
Epoch 3/40
32/32 [==============================] - ETA: 0s - loss: 0.0115 - mae: 0.0106 -

In [25]:
model = load_model('/content/drive/MyDrive/0_Hasnat/2_IEEE Transaction Industrial Informatics/Hybrid Model/CheckpointsE3/E2t3-cp-0040-loss0.01.h5')
y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y.reshape(-1, 1))
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled))
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 217.0
Median Absolute Error (MedAE): 169.24
Mean Squared Error (MSE): 83854.33
Root Mean Squared Error (RMSE): 289.58
Mean Absolute Percentage Error (MAPE): 2.12 %
Median Absolute Percentage Error (MDAPE): 1.64 %


y_test_unscaled.shape=  (4600, 1)
y_pred.shape=  (4600, 1)
